In [5]:
### 方法参考来之：https://www.cnblogs.com/always-fight/p/10159547.html
## 2019-11-3

In [1]:
import sys
sys.version

'3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]'

In [12]:
import numpy as np
import pandas as pd
import re
import jieba
import warnings 
warnings.filterwarnings('ignore')
from gensim.models import Word2Vec

In [40]:
df_technology = pd.read_csv("./NLPData/technology_news.csv",encoding ="utf-8")

In [41]:
df_technology.head()

,Unnamed: 0,content
0,0,",中新网,1月7日电 恰逢CES 2017拉开大幕，却惊闻“AlphaGo升级版”的M..."
1,1,徐立，商汤科技CEO在谈起本次参展时谈到：“作为一个成立刚刚两年的创业公司，这次参展，一...
2,2,正如最后挑战Master的古力在落败后发表的观点：“人类与人工智能共同探索围棋世界的大幕...
3,3,", SenseFace人脸布控的“黑科技”"
4,4,本届大展最大的看点，无疑是“被誉为2016全美科技第一神股”英伟达的首次CES主题演讲。...


### 1.加载数据

In [22]:
import pandas as pd
# df_technology
df_technology  = pd.read_csv("./NLPData/technology_news.csv", encoding = 'utf-8')
df_technology  = df_technology.dropna() # 取消可能为空的行数
# sprots
df_sprots   = pd.read_csv("./NLPData/sports_news.csv", encoding = 'utf-8')
df_sprots   = df_sprots.dropna()

#df_society
df_society = pd.read_csv("./NLPData/society_news.csv",encoding = 'utf-8')
df_society = df_society.dropna()

#military
df_military   = pd.read_csv("./NLPData/military_news.csv", encoding = 'utf-8')
df_military   = df_military.dropna()

df_internality = pd.read_csv("./NLPData/international_news.csv",encoding = 'utf-8')
df_international = df_internality.dropna()

df_house = pd.read_csv("./NLPData/house_news.csv",encoding = 'utf-8')
df_house = df_house.dropna()

df_home = pd.read_csv("./NLPData/home_news.csv",encoding = 'utf-8')
df_home = df_home.dropna()

df_finance = pd.read_csv("./NLPData/finance_news.csv",encoding = 'utf-8')
df_finance = df_finance.dropna()

# df_entertainment 
df_entertainment   = pd.read_csv("./NLPData/entertainment_news.csv", encoding = 'utf-8')
df_entertainment   = df_entertainment.dropna()

# df_car
df_car  = pd.read_csv("./NLPData/car_news.csv", encoding = 'utf-8')
df_car  = df_car.dropna()

In [7]:
print (df_car.content.values.tolist()[11778])

　　神州专车高考生免费送考公益活动已持续三年，今年将覆盖30余城市，预计服务数千名考生。神州专车自2015年初在全国60大城市上线，以“专业司机、专业车辆”模式，为用户提供安全、舒适、标准化的出行服务。根据罗兰贝格报告，神州专车占据40%的专车市场份额，消费者满意度和安全评分均位居行业首位。


In [23]:
### stopwords ###
stopword_list = [k.strip() for k in open("./data/stopwords_NLP.txt",encoding ='utf-8') if k.strip()!= '']

### 1.1下面的代码用来采集全部的样本数据

In [13]:
# 读取文件里面的全部的csv文件 ###
import os
import csv
import pandas as pd
import sqtime

cutWords = []
example  = 0
t1 = time.time()
for root,dirs,files in os.walk("./NLPData"):
    for file in files:
        if os.path.splitext(file)[1] == '.csv':
            label    = os.path.splitext(file)[0] # 以文件的名字做为标签区分 #
            doc      = pd.read_csv(os.path.join(root,file),encoding = 'utf-8')
            contents = doc.content.dropna()
            for each_line in contents:
                if len(each_line)>20:
                    example +=1
                    cutWord = [word for word in jieba.lcut(each_line) if (word not in stopword_list and (word.strip()!= ''or word.lstrip()!=''))]
                    cutWords.append([" ".join(cutWord),label])
                    if example%10000 == 0:
                        t2 = time.time()
                        print("处理完前%d个样本所花费的时间是%0.2f"%(example,(t2 - t1)/60),'mins')

### 1.2采集超过特定长度，数量相同的个分类样本

In [24]:
import time
t1 = time.time()
def read_files(file,cutWords,label,limit,maxs,i=0):
    doc     = pd.read_csv(file,encoding = 'utf-8')
    content = doc.content.dropna()
    for each_line in content:
        if len(each_line)>limit:
            i = i+1
            if i<=maxs:
                cutWord = [word for word in jieba.lcut(each_line) if (word not in stopword_list and (word.strip()!= ''or word.lstrip()!=''))]
                cutWords.append([" ".join(cutWord),label])
            if i%40000 == 0:
                print("处理完前%d个样本所花费的时间是%0.2f"%(i,(time.time()- t1)/60),'mins')

In [36]:
#calling 
import os
limit = 100     #采集的样本长度大于这个值
maxs  = 15000  # 每个总类的样本数量都是maxs 
cutWords = []
for root,dirs,files in os.walk("./NLPData"):
    for file in files:
        if os.path.splitext(file)[1] == '.csv':
            label    = os.path.splitext(file)[0] # 以文件的名字做为标签区分 #
            file     = os.path.join(root,file)
            read_files(file,cutWords,label,limit,maxs)

处理完前40000个样本所花费的时间是18.12 mins
处理完前40000个样本所花费的时间是18.95 mins
处理完前40000个样本所花费的时间是21.14 mins
处理完前80000个样本所花费的时间是21.14 mins


### 1.3 保持分词后的结果

In [38]:
### 先要乱序
import random
random.shuffle(cutWords)

In [39]:
### 将cutWords 写入local文件##
with open("./NLPData/cutWords_lists1.txt",'w',encoding = 'utf-8') as file:
    for cutword in cutWords:
        if (cutword[0]!=''and len(cutword[0])>20):# 文本为空的情况，跳过
            file.write(''.join(cutword[0])+'\n')#cutword[0] ->content
file.close()

In [40]:
### 获得文本的对应标签写入单独的文件#
with open("./NLPData/cutWords_labels1.txt",'w',encoding = 'utf-8') as file:
    for cutword in cutWords:
        if (cutword[0]!=''and len(cutword[0])>20):  # 文本为空的情况，跳过
            file.write(''.join(cutword[1])+'\n')#cutword[1] ->label
file.close()

### 1.3.1 去掉文本中的空行

In [111]:
def del_blank(file1,file2):
    file1 = open(file1, 'r', encoding='utf-8') # 要去掉空行的文件 
    file2 = open(file2, 'w', encoding='utf-8') # 生成没有空行的文件
    try:
        for line in file1.readlines():
            if line == '\n':
                line = line.strip("\n")
            file2.write(line)
    finally:
        file1.close()
        file2.close()

#del_blank("./NLPData/cutWords_lists1.txt","./NLPData/cutWords_lists2.txt")

### 1.4重新加载文本序列

In [85]:
cutWords_list = []
with open("./NLPData/cutWords_lists1.txt",'r',encoding = 'utf-8') as file:
    cutWords_list = [k.split() for k in file]

In [86]:
with open("./NLPData/cutWords_labels1.txt",'r',encoding = 'utf-8') as file:
    cutWords_labels = [k.strip() for k in file]

In [87]:
len(cutWords_list),len(cutWords_labels),cutWords_labels[78000]

(123243, 123243, 'sports_news')

### 2 Word2vec模型训练过程 - 16000次花费时间8++hours

In [44]:
import time
import warnings 
warnings.filterwarnings('ignore')
from gensim.models import Word2Vec

### 155/5 =31 mins,每1000次迭代需要0.5hours ###
iters = 1000*20
start = time.time()
word2vec_model  = Word2Vec(cutWords_list,size = 100,window = 4,iter = iters,min_count = 20)
end = time.time()
print("训练%d个样本,迭代%d次花费的时间是%0.2f"%(len(cutWords_list),iters,(end - start)/60),'mins')
## save model ###
word2vec_model.save("./NLPData/word2vec_text_classifier_model_one.w2v")

训练123244个样本,迭代20000次花费的时间是1331.68 mins


In [48]:
1331.68/60

22.194666666666667

In [45]:
## 简单相似度计算 ### 
word2vec_model.wv.most_similar("汽车",topn = 4)

[('汽车产业', 0.6975545287132263),
 ('电动汽车', 0.69080650806427),
 ('汽车行业', 0.6847970485687256),
 ('车', 0.6402374505996704)]

In [13]:
## 简单相似度计算 ### 
word2vec_model.wv.most_similar("汽车",topn = 4)

[('电动汽车', 0.6200621128082275),
 ('乘用车', 0.594478964805603),
 ('车', 0.5929100513458252),
 ('电动车', 0.558305025100708)]

In [47]:
## save model ###
word2vec_model.save("./NLPData/word2vec_text_classifier_model.w2v")

In [46]:
from gensim.models import Word2Vec
word2vec_models = Word2Vec.load("./NLPData/word2vec_text_classifier_model.w2v")
word2vec_models.wv.most_similar('科技',topn = 5)

[('技术', 0.5855147838592529),
 ('高科技', 0.5632357597351074),
 ('信息技术', 0.4843491017818451),
 ('技术创新', 0.4782775640487671),
 ('能源技术', 0.45032164454460144)]

In [69]:
### 重新加载模型
from gensim.models import Word2Vec
word2vec_models = Word2Vec.load("./NLPData/word2vec_text_classifier_model.w2v")
word2vec_models.wv.most_similar('科技',topn = 5)

[('技术', 0.5855147838592529),
 ('高科技', 0.5632357597351074),
 ('信息技术', 0.4843491017818451),
 ('技术创新', 0.4782775640487671),
 ('能源技术', 0.45032164454460144)]

### 3 特征工程 - 找到每个词对应的词向量

In [49]:
### 对于每一篇文章，获取文章的每个分词在word2vec模型中的向量；
### 然后每一篇文章中词分词都可以在word2vec模型中的
### 相关性向量的求和后的平均数，即此篇文章在word2vec模型中的相关向量
def getVector(cutWords,word2vec_models):
    count = 0
    article_vector = np.zeros(word2vec_models.layer1_size) #word2vec_models.layer1_size = 100
    for cutWord in cutWords:
        if cutWord in word2vec_models:
            article_vector+= word2vec_models[cutWord]
            count+=1 
    return article_vector/count # 返回的是文档中每个词对应词向量的平均值
# 这里对每一行的样本中每个词的都找到词向量，然后求词向量的评价后作为这个样本的向量表示

In [88]:
# 取一部分数量的词来更换成词向量
import time
cutword_nums = 123243
start        = time.time()
vector_list  = []
i = 0           
for cutWords in cutWords_list[:cutword_nums]:
    vector_list.append(getVector(cutWords,word2vec_models))
    i+=1
    if i%5000 == 0:
        print("前%d篇文章所形成词向量花费的时间%0.2f" %(i,time.time() - start),'sec')
X = np.array(vector_list)
print("处理全部花费的时间:%0.2f"%(time.time() - start),'sec')

前5000篇文章所形成词向量花费的时间2.31 sec
前10000篇文章所形成词向量花费的时间4.57 sec
前15000篇文章所形成词向量花费的时间6.79 sec
前20000篇文章所形成词向量花费的时间9.07 sec
前25000篇文章所形成词向量花费的时间11.38 sec
前30000篇文章所形成词向量花费的时间13.65 sec
前35000篇文章所形成词向量花费的时间15.89 sec
前40000篇文章所形成词向量花费的时间18.19 sec
前45000篇文章所形成词向量花费的时间20.42 sec
前50000篇文章所形成词向量花费的时间22.75 sec
前55000篇文章所形成词向量花费的时间25.10 sec
前60000篇文章所形成词向量花费的时间27.40 sec
前65000篇文章所形成词向量花费的时间29.62 sec
前70000篇文章所形成词向量花费的时间31.95 sec
前75000篇文章所形成词向量花费的时间34.29 sec
前80000篇文章所形成词向量花费的时间36.59 sec
前85000篇文章所形成词向量花费的时间38.89 sec
前90000篇文章所形成词向量花费的时间41.19 sec
前95000篇文章所形成词向量花费的时间43.41 sec
前100000篇文章所形成词向量花费的时间45.58 sec
前105000篇文章所形成词向量花费的时间47.77 sec
前110000篇文章所形成词向量花费的时间49.95 sec
前115000篇文章所形成词向量花费的时间52.30 sec
前120000篇文章所形成词向量花费的时间54.54 sec
处理全部花费的时间:56.06 sec


In [51]:
X[0],len(X[0])

(array([ 1.21009024,  3.4342918 ,  1.08379322, -1.28321916, -0.66442199,
         2.93336204, -3.82177398,  2.05418622, -0.4823567 , -1.29134928,
         0.9407235 ,  2.95109658,  2.70000585,  0.16573589,  2.69477381,
         0.62821775, -0.85666412, -0.43557714,  0.16148297, -0.95581335,
         1.283366  , -0.6646044 , -2.43047636, -2.55841866,  0.21417034,
        -3.72447912, -0.5029494 , -1.51254707, -0.38377427, -0.73080978,
         0.40184854, -0.43851835, -0.72480179,  1.7313887 , -0.10916261,
         0.01783772,  1.80194649, -0.38351465,  1.17623264,  0.49000483,
        -1.26437074,  0.67221092, -0.63962285, -0.56758915, -1.3795338 ,
        -1.27411467, -1.81222239,  0.91498358, -1.91736301,  2.36589533,
         0.14607058, -1.55139714, -2.17481818, -1.1852677 , -2.65657296,
        -2.98212741,  1.21296303, -1.63076975, -1.22166217, -3.41532409,
        -1.74267106,  2.02215778, -3.41850342,  0.92721958,  1.66589735,
         1.06933595,  0.06480429, -3.62354255, -1.2

In [90]:
### j将计算的特征矩阵save成文档来
X.dump('./NLPData/articles_vector.txt') # dump方法传入一个字符串

In [30]:
import numpy as np
X = np.load('./NLPData/articles_vector.txt',allow_pickle = True)

### 4. 将label 转成数字标签

In [91]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
labelencoder = LabelEncoder()
with open("./NLPData/cutWords_labels1.txt",'r',encoding = 'utf-8') as file:
    y_labels = [k.strip() for k in file]
y =  labelencoder.fit_transform(y_labels[:cutword_nums]) # 所有的label y

In [92]:
len(X),len(y),type(X),type(y)

(123243, 123243, numpy.ndarray, numpy.ndarray)

### 5 数据样本分割 

### 5.1样本分割前线判断X或者y里面是不是用空的值

In [89]:
np.isnan(X).any(),np.isnan(y).any()

(False, False)

In [68]:
X[np.isnan(X) == True] = 0  # 将空的值强制为零

In [84]:
for i in range(len(X)):
    if np.isnan(X[i]).any() == True:
        print('i = ',i)

i =  79617


### 5.2 样本数据分割

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 200)
log_clf = LogisticRegression(multi_class = 'multinomial',solver = 'lbfgs') # ,
log_clf.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [164]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 200)
log_clf = LogisticRegression()
log_clf.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [94]:
print("逻辑回归模型准确率:%0.2f"%(log_clf.score(X_test,y_test)*100),"%")

逻辑回归模型准确率:73.93 %


In [37]:
### 通过joblib来保存model
from sklearn.externals import joblib
joblib.dump(log_clf,'./NLPData/logistic.model')

['./NLPData/logistic.model']

In [38]:
### 加载model
from sklearn.externals import joblib
logistic_model = joblib.load('./NLPData/logistic.model')

### 5.3 相同的模型使用交叉验证查看准确率

In [39]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
cv_split        = ShuffleSplit(X.shape[0],train_size = 0.7,test_size = 0.2) # X.shape[0] = 24000  #n_iter default = 10
log_model       = LogisticRegression(multi_class='multinomial', solver='lbfgs')
score_ndarray   = cross_val_score(log_model,X,y,cv = cv_split)
print(score_ndarray)
print("交叉验证的准确率是%0.2f"%(score_ndarray.mean()*100),"%")

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[ 0.92233333  0.919       0.92633333  0.92183333  0.92316667  0.92733333
  0.919       0.92066667  0.921       0.92216667]
交叉验证的准确率是92.23 %


### ==============另一种方法:采用TF-IDF算法提取特征=================###

### 1 重新加载样本和label

In [2]:
cutWords_list = []
with open("./NLPData/cutWords_lists1.txt",'r',encoding = 'utf-8') as file:
    cutWords_list = [k.split() for k in file]

In [3]:
cutWords_labels = []
with open("./NLPData/cutWords_labels1.txt",'r',encoding = 'utf-8') as file:
    cutWords_labels = [k.strip() for k in file]

### 2 通过Tf-idf vectorize 来抽取特征

In [6]:
### stopwords ###
stopword_list = [k.strip() for k in open("./data/stopwords_NLP.txt",encoding ='utf-8') if k.strip()!= '']
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf         = TfidfVectorizer(cutWords_list,stop_words =stopword_list,min_df = 50,max_df = 0.3)

In [ ]:
import time
t1 = time.time()
def read_files(file,cutWords,label,limit,maxs,i=0):
    doc     = pd.read_csv(file,encoding = 'utf-8')
    content = doc.content.dropna()
    for each_line in content:
        if len(each_line)>limit:
            i = i+1
            if i<=maxs:
                
            if i%20000 == 0:
                print("处理完前%d个样本所花费的时间是%0.2f"%(i,(time.time()- t1)/60),'mins')

In [4]:
doc         = pd.read_csv("./NLPData/car_news.csv",encoding = 'utf-8')
doc.columns = ['index','text']
i = 0
for one in doc['text']:
    i+=1
    if i ==1:
        ones = "/".join(one)
        print(type(ones))
        print(ones)

<class 'str'>
　/　/本/报/济/南/1/月/2/日/电/ /(/记/者/潘/俊/强/)/日/前/，/备/受/关/注/的/济/南/“/专/车/案/”/一/审/宣/判/。/济/南/市/市/中/区/人/民/法/院/对/济/南/市/民/陈/超/诉/济/南/市/城/市/公/共/客/运/管/理/服/务/中/心/行/政/处/罚/一/案/作/出/一/审/宣/判/，/判/决/撤/销/济/南/市/客/管/中/心/对/“/专/车/”/司/机/陈/超/的/行/政/处/罚/。


In [55]:
doc['text'][2]

'\u3000\u3000济南市市中区人民法院审理认为，本案中，陈超在与乘客通过网络约车软件取得联系后，使用未取得运营证的车辆将乘客送至目的地，并按约定收取了车费。陈超的行为构成未经许可擅自从事出租汽车客运经营，违反了现行法律的规定。但虑及网约车这种共享经济新业态的特殊背景，该行为的社会危害性较小。'

In [48]:
### 特征工程-抽取特征### 
type(doc)

pandas.core.frame.DataFrame

In [115]:
### 读取特定path下文件夹里面的特定格式的文件，并写入特定文件内###
import os 
import csv 
import pandas as pd
def read_file(path,file,special_format):
    writer = open(path,'w',encoding = 'utf-8')
    for root,dirs,files in os.walk():
        for file in files:
            if os.path.splitext(file)[1] == special_format:
                with open(os.path.join(root,file),'r',encoding = "utf-8") as f:
                    for line in f.readlines():
                        writer.write(line.strip('\n')+'\n')
    writer.close()
#Example:
#file = "./datas/mt/酒店评论数据备份v2/数据分割处理/postive/content/contentMergedPostive.txt"
#path = "./datas/mt/酒店评论数据备份v2/数据分割处理/postive/content"
#special_format = ".txt"
#read_file(path,file,special_format)